In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dat-a-thon23/sample_submission.csv
/kaggle/input/dat-a-thon23/train.csv
/kaggle/input/dat-a-thon23/test.csv


In [2]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [3]:
df=pd.read_csv('/kaggle/input/dat-a-thon23/train.csv')
df.head()

,id,categoryA,categoryB,categoryC,categoryD,categoryE,categoryF,featureA,featureB,featureC,...,compositionC,compositionD,compositionE,compositionF,compositionG,compositionH,compositionI,compositionJ,unit,result
0,a563699ca2a601c6ac64aa29986a00a90fb42b48741695...,catA_1,catB_0,catC_718,catD_0,catE_0,catF_0,75808.375,4.457840,0.005718,...,26.0,0.0,32.0,3.0,0.0,9.0,22.26,20.0,unit_6,0.000458
1,91ab3eb3bcf6c8c1c5fe2da9ba671aa5a48c7369d9a50f...,catA_1,catB_0,catC_1309,catD_0,catE_0,catF_0,75808.375,4.457840,0.005718,...,26.0,0.0,32.0,3.0,0.0,9.0,22.26,20.0,unit_6,0.000335
2,7128c51c554735d6c81862684ad6005ae12d2edbcd4644...,catA_15,catB_0,catC_1309,catD_0,catE_0,catF_0,75808.375,4.457840,0.005718,...,26.0,0.0,32.0,3.0,0.0,9.0,22.26,20.0,unit_4,0.054072
3,c8144b52e4f63014de0a0d8e1c629bf0b05cb2696cfc23...,catA_0,catB_0,catC_935,catD_0,catE_0,catF_0,75808.375,4.457840,0.005718,...,26.0,0.0,32.0,3.0,0.0,9.0,22.26,20.0,unit_5,0.061143
4,88d15a5b2df6692f23d105ff1ae82ae026be00c9271eef...,catA_22,catB_0,catC_1325,catD_0,catE_2,catF_0,-40055.250,4.363288,0.729194,...,14.0,0.0,49.0,3.0,2.0,9.0,16.84,15.0,unit_15,0.015439


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13000 entries, 0 to 12999
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            13000 non-null  object 
 1   categoryA     13000 non-null  object 
 2   categoryB     13000 non-null  object 
 3   categoryC     13000 non-null  object 
 4   categoryD     13000 non-null  object 
 5   categoryE     13000 non-null  object 
 6   categoryF     13000 non-null  object 
 7   featureA      12936 non-null  float64
 8   featureB      12936 non-null  float64
 9   featureC      12936 non-null  float64
 10  featureD      12936 non-null  float64
 11  featureE      12936 non-null  float64
 12  featureF      13000 non-null  float64
 13  featureG      12936 non-null  float64
 14  featureH      12936 non-null  float64
 15  featureI      12936 non-null  float64
 16  compositionA  12936 non-null  float64
 17  compositionB  12936 non-null  float64
 18  compositionC  12936 non-nu

In [5]:
df.shape

(13000, 28)

In [6]:
df=df.dropna()
df.shape

(12936, 28)

In [7]:
# Separate the features and target variable
X = df.drop('result', axis=1)
y = df['result']*100

In [8]:
# Convert categorical variables into numerical variables using label encoding
le = LabelEncoder()
X_categorical = X.select_dtypes(include=['object'])
X_encoded = X_categorical.apply(le.fit_transform)
X_numeric = X.select_dtypes(exclude=['object'])
X_encoded = pd.concat([X_encoded, X_numeric], axis=1)

In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12936 entries, 0 to 12999
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            12936 non-null  object 
 1   categoryA     12936 non-null  object 
 2   categoryB     12936 non-null  object 
 3   categoryC     12936 non-null  object 
 4   categoryD     12936 non-null  object 
 5   categoryE     12936 non-null  object 
 6   categoryF     12936 non-null  object 
 7   featureA      12936 non-null  float64
 8   featureB      12936 non-null  float64
 9   featureC      12936 non-null  float64
 10  featureD      12936 non-null  float64
 11  featureE      12936 non-null  float64
 12  featureF      12936 non-null  float64
 13  featureG      12936 non-null  float64
 14  featureH      12936 non-null  float64
 15  featureI      12936 non-null  float64
 16  compositionA  12936 non-null  float64
 17  compositionB  12936 non-null  float64
 18  compositionC  12936 non-nu

In [10]:
# Scale the numerical variables using standardization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

# Convert the target variable into categorical variable using one-hot encoding
y_categorical = to_categorical(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42)

# Define the model
model = Sequential()
model.add(Dense(64, input_dim=X_scaled.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_categorical.shape[1], activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
early_stop = EarlyStopping(monitor='val_loss', patience=10)
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stop])

# Evaluate the model on the testing dataset
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]:.4f}')
print(f'Test accuracy: {score[1]:.4f}')

Epoch 1/50
259/259 [==============================] - 2s 4ms/step - loss: 3.4340 - accuracy: 0.3140 - val_loss: 3.0207 - val_accuracy: 0.3498
Epoch 2/50
259/259 [==============================] - 1s 4ms/step - loss: 3.0517 - accuracy: 0.3376 - val_loss: 2.9221 - val_accuracy: 0.3502
Epoch 3/50
259/259 [==============================] - 1s 4ms/step - loss: 2.9485 - accuracy: 0.3404 - val_loss: 2.8415 - val_accuracy: 0.3498
Epoch 4/50
259/259 [==============================] - 1s 4ms/step - loss: 2.8889 - accuracy: 0.3422 - val_loss: 2.8074 - val_accuracy: 0.3498
Epoch 5/50
259/259 [==============================] - 1s 4ms/step - loss: 2.8430 - accuracy: 0.3402 - val_loss: 2.7731 - val_accuracy: 0.3498
Epoch 6/50
259/259 [==============================] - 1s 4ms/step - loss: 2.8225 - accuracy: 0.3405 - val_loss: 2.7620 - val_accuracy: 0.3507
Epoch 7/50
259/259 [==============================] - 1s 4ms/step - loss: 2.8051 - accuracy: 0.3417 - val_loss: 2.7432 - val_accuracy: 0.3512
Epoch 

In [11]:
# Load the new data into a pandas dataframe
new_df = pd.read_csv('/kaggle/input/dat-a-thon23/test.csv')

# Convert categorical variables into numerical variables using label encoding
X_categorical_new = new_df.select_dtypes(include=['object'])
X_encoded_new = X_categorical_new.apply(le.fit_transform)
X_numeric_new = new_df.select_dtypes(exclude=['object'])
X_encoded_new = pd.concat([X_encoded_new, X_numeric_new], axis=1)

# Scale the numerical variables using standardization
X_scaled_new = scaler.transform(X_encoded_new)

# Predict the water quality result on the new dataset
y_pred = model.predict(X_scaled_new).argmax(axis=1).astype(np.float32)
# y_pred = model.predict(X_scaled_new)
# y_pred = np.argmax(y_pred, axis=1)

# Add the predicted water quality result column to the original dataframe
new_df['result'] = y_pred/100

output_data = new_df[['id', 'result']]

# Save the updated dataframe to a new csv file
output_data.to_csv('water_quality_data_with_predictions.csv', index=False)

219/219 [==============================] - 1s 2ms/step


In [12]:
output_data.head(20)

,id,result
0,939edadbc8e0cddf21c8c5710c4f2f909abd36c196aee8...,0.01
1,fcfed000585350fdc6a982bd1146cd8cdd71d989a2fb01...,0.02
2,917b412b73b7c1c83f779d2f350acda4d2b14804579e79...,0.00
3,012d646dbd712fd4b752aa8761dc18aaf28643c262f859...,0.00
4,77c69c2a873444dfef22131bada7d9504ad731ca8df55d...,0.00
5,547afb1ac49f47e228912d5694e2703fab5007a06d165f...,0.03
6,bfd22324ba43c7392f9059944b4eb1a03a58e14347f4cf...,0.00
7,6bb430339d5ee94426a414737b08fbecea9d9e600d563a...,0.00
8,522ffe87e4f684d3b7108976d28053c8c51e02784d7cda...,0.00
9,f2742f7f09893e8f6bb1ddc1c54726cb775713eab3a5e7...,0.00


In [13]:
output_data.shape

(7000, 2)